In [ ]:
#Import

import matplotlib.pyplot as plt
import matplotlib as mt
import csv
import pickle
import sys
import dlib
import cv2
import numpy as np
import os

In [ ]:
#Leggo il file csv nomecsv.csv lo salvo in etic

etic=[]
with open('nomecsv.csv', newline='') as csvfile:
    etichetta2 = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in etichetta2:
        etic.append(row)

#Prendo i path relativi.
#etic[n][2] significa che prendo la casella numero 2 della riga, ovvero il path.
#CSV come quelli di Chalearn non hanno informazioni sul gender, quindi hanno 2 e non 3 celle per riga.

filenames=[]
leng=len(etic)
for n in range (0,leng):
    filenames.append(etic[n][2])

In [ ]:
#Questi sono dei parametri per "tagliare" una immagine con margine 40%.

x=51
yy=51
w=154
h=154
r = max(w, h) / 2
centerx = x + w/ 2
centery = yy + h/ 2
nx = int(centerx - r)
ny = int(centery - r)
nr = int(r * 2)

In [ ]:
#Qui creo gli hog.

#Definizione dell'hog.
hog = cv2.HOGDescriptor((128,128),(32,32),(16,16),(16,16),9)

#Dei parametri di supporto
X_hog=[]
y=[]
n=0
img=None
gray_img=None
h=None
hogghy=np.zeros(1764)

#Ciclo il path relativo, carico l'immagine, la porto in scala di grigi, la taglio, ne faccio la resize a 128x128 e ne calcolo l'hog.
for images in filenames:
    img = cv2.imread(images)
    
    if img is not None:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faceimg = gray_img[ny:ny+nr, nx:nx+nr]
        resized = cv2.resize(faceimg, (128, 128))
        
        h = hog.compute(resized)
        
        for i in range(0,len(h)):
            hogghy[i]=h[i][0]
            
        X_hog.append(hogghy)
        
        #Nota che qui prendo solo il gender ( la prima cella della riga )
        y.append(int(etic[n][0]))
    n=n+1

In [ ]:
#Salvo hog ed etichette in un file pkl .

#with open("hog.pkl","wb") as output:
#        pickle.dump(X_hog,output)
#with open("etichetta.pkl","wb") as output:
#    pickle.dump(y,output)

In [ ]:
#Salvo i volti in un file pkl .

with open("nomehog.pkl","wb") as output:
        pickle.dump(X_hog,output)


with open("nomeetichetta.pkl","wb") as output:
    pickle.dump(y,output)

In [ ]:
#UN ESEMPIO DI ADDESTRAMENTO CON VGG

In [ ]:
#Tutti gli import

import random
import deap
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_deap_master.evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
#Qui leggo i pickle.
#In Hog metto gli hog, in y le etichette.

with open("nomehog.pkl","rb") as data:
    Hog= pickle.load(data)
with open("nomeetichetta.pkl","rb") as data:
    y = pickle.load(data)

In [ ]:
#Inizalizzo dei vettori, è una operazione non utile in questo caso

X_train_hog=Hog

y_train_1=np.zeros(len(y))
k=0
for i in range(0,len(y)):
    y_train_1[i]=y[k]
    k=k+1

#Nel caso in cui le etichette presentano la virgola
Y_train_1=[]
for n in range(0,len(y_train_1)):
    Y_train_1.append(int(y_train_1[n]))

print("-----")

In [ ]:
#Addestro.

svc = svm.SVC(kernel='linear', C=0.17782, verbose=True)
svc.fit(X_train_hog,Y_train_1)

In [ ]:
#Salvataggio in pkl del classificatore con joblib.
from sklearn.externals import joblib
joblib.dump(svc,'svc_vgg_linear.pkl')

In [ ]:
#Calcolo dello score

mscore=svc.score(X_train_hog, Y_train_1)
print(mscore)